In [1]:
import os
import json
from datetime import datetime,date
import pickle
import sqlite3


In [2]:
def strQ2B(s):
    # 全形轉半形
    new_string = ""
    for char in s:
        inside_code=ord(char) # 轉unicode
        if inside_code == 12288:                              #全形空格直接轉換            
            inside_code = 32 
        elif (inside_code >= 65281 and inside_code <= 65374): # 轉換其他全形字
            inside_code -= 65248
        new_string += chr(inside_code)
    return new_string
print(strQ2B('ＨＩ，ＯＫＯＫ'))

HI,OKOK


In [3]:
reviews_dir = 'yahoo_movie_review/'
directory = os.fsencode(reviews_dir)

review_paths = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".json"): 
        review_paths.append(os.path.join(reviews_dir, filename))

movie_list = []
for rpath in review_paths:
    try:
        with open(rpath, 'r') as fin:
            movie = json.load(fin, encoding='utf-8')
    except:
        continue
    movie_list.append(movie)
print(len(movie_list))
print(movie_list[0])

4550
{'length': '1時51分', 'director': '馬克威廉斯(Mark Williams)', 'm_id': 6827, 'c_title': '愛在回家時', 'review': [{'time': '2017-05-27', 'title': '《愛在回家時》回家吧，回到最初的美好。完整觀後推薦心得', 'content': '《愛在回家時》完整觀後推薦心得：http://0rz.tw/IUhKW (痞客邦)', 'rate': 5, 'author': '迪兒'}], 'date': '2017-05-26', 'img': 'https://s.yimg.com/vu/movies/fp/mpost/68/27/6827.jpg', 'e_title': 'A Family Man', 'cast': '《全面攻佔系列》傑瑞德巴特勒(Gerard Butler)、《捍衛任務》威廉達佛(Willem Dafoe)、《單身啪啪啪》愛莉森布里(Alison Brie)、《海邊的曼徹斯特》珂茜莫爾(Gretchen Mol)、《蜘蛛人2》艾爾菲摩里納(Alfred Molina)', 'category': '劇情、溫馨/家庭'}


In [4]:
with open('data/mid_to_cidofsent.bin','rb') as fin:
    mid_to_cidofsent = pickle.load(fin)
    
with open('data/mid_to_tags.bin','rb') as fin:
    mid_to_tags= pickle.load(fin)

In [5]:
print(mid_to_tags['2160'])

[('知道 演', 0), ('個人 觀感', 1), ('看過 小說', 2), ('呎 吋', 3), ('票房 成績', 4), ('系列 電影', 5), ('交代 清楚', 6), ('希望 下', 7)]


In [6]:
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

In [7]:
#insert movie

sql_m = "insert into reviews_movie (movie_id, title, title_original, director, cast, image, length, review_count, date  ) values (?, ?, ?, ?,?, ?, ?, ?, ?)"

for movie in movie_list:
    try:
        m_id = movie['m_id']
#         mobj = (m_id, movie['c_title'],movie['e_title'], movie['director'], movie['cast'], movie['img'], movie['length'], len(movie['review']), datetime.strptime(movie['date'], '%Y-%m-%d'))
        mobj = (m_id, movie['c_title'],movie['e_title'], movie['director'], movie['cast'], movie['img'], movie['length'], len(movie['review']), movie['date'])

        cursor.execute(sql_m, mobj)
    except Exception as e:
        print(str(e))
        print(movie['c_title'])



In [8]:
#insert review

sql_r = "insert into reviews_review (movie_id, rate, author, title, content, date ) values (?, ?,?, ?, ?, ?)"

i=0
for movie in movie_list:
    m_id = movie['m_id']
    if len(movie['review']) > 149:
        i+=1
        for review in movie['review']:
            robj = (m_id, review['rate'], review['author'], review['title'], review['content'], review['time'])
    #       robj_list.append(roj)
            cursor.execute(sql_r, robj)
print(i)


433


In [9]:
cursor.execute("select * from reviews_movie")
result = cursor.fetchall()
print(len(result))


4550


In [10]:
#insert tag

sql_t = "insert into reviews_tag (movie_id, name, cluster_id ) values (?, ?,?)"
for movie_id, tags in mid_to_tags.items():
    for tag in tags:
        tobj = (movie_id, tag[0], int(tag[1]))
        cursor.execute(sql_t, tobj)

In [11]:
#insert sentence
with open('data/mid_to_reviews_in_sents.json', 'r') as fin:
    mid_to_reviews_in_sents = json.load(fin)

mid_to_reviews_info = {}
for movie in movie_list:
    mid_to_reviews_info[movie['m_id']] =  movie['review']
    
sql_s = "insert into reviews_sentence (review_id_id, content, cluster_id, review_position) values (?, ?, ?, ?)"

review_id = 1
for mid in mid_to_reviews_in_sents:
    review_sents_lst = mid_to_reviews_in_sents[mid]
    review_cid_lst = mid_to_cidofsent[mid]
    review_info_lst = mid_to_reviews_info[int(mid)]
    for  review_sents, review_cids, review_info in zip(review_sents_lst, review_cid_lst, review_info_lst):
        review_raw = strQ2B(review_info['content'])
        for sent, cid in zip(review_sents, review_cids):
            position = review_raw.find(sent)
            if position == -1:
                print(sent)
                print(review_raw)
            sobj = (review_id, sent, int(cid), position)
            cursor.execute(sql_s, sobj)
        review_id+=1
        
            

In [12]:
conn.commit()

In [13]:
conn.close()